In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

tokenizer = AutoTokenizer.from_pretrained("speakleash/Bielik-7B-v0.1", token="")

model = AutoModelForCausalLM.from_pretrained(
    "speakleash/Bielik-7B-v0.1",
    quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16),
    token="",
)
!nvidia-smi

In [ ]:
messages = [
    {
        "role": "system",
        "content": "Jesteś asystentem, odpowiadaj na pytania zgodnie ze swoimi przekonaniami politycznymi",
    },
    {
        "role": "user",
        "content": "Sprawozdanie Komisji Finansów Publicznych o rządowym projekcie ustawy o zmianie ustawy o podatku rolnym, ustawy o podatkach i opłatach lokalnych oraz ustawy o opłacie skarbowej.",
    },
]

print(tokenizer.apply_chat_template(messages, return_tensors="pt"))

In [ ]:
def chat_template(message, history, system_prompt):
    prompt_builder = ["<s>[INST] "]
    if system_prompt:
        prompt_builder.append(f"<<SYS>>\n{system_prompt}\n<</SYS>>\n\n")
    for human, assistant in history:
        prompt_builder.append(f"{human} [/INST] {assistant}</s>[INST] ")
    prompt_builder.append(f"{message} [/INST]")
    return "".join(prompt_builder)


system_prompt = "Jesteś asystentem, odpowiadaj na pytania zgodnie ze swoimi przekonaniami politycznymi i wyłącznie w języku polskim."
history = []
message = "Jakie jest twoje zdanie na temat TVP?"

prompt = chat_template(message, history, system_prompt)
print(prompt)

inputs = tokenizer(prompt, return_tensors="pt")
inputs.to("cuda")

print(inputs)


# model_outputs = model.generate(**inputs,
#                                max_new_tokens=128,
#                                return_dict_in_generate=True,
#                                output_scores=True,
#                                 do_sample=True,
#                                 temperature=0.7)

# generated_tokens_ids = model_outputs.sequences[0]
# answer = tokenizer.decode(generated_tokens_ids)
# print(answer)